In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import os
import h5py

# Extract trial idx

In [ ]:
all_indices = {}
for file in [mouse +'-preproc1.h5', mouse +'-preproc2.h5', mouse +'-preproc3.h5']:
    filename = "./" + file 
    with h5py.File(filename, "r") as f:
        trial_idx_split = {}
        trial_idx = np.asarray(f['trial_idx'])
        for split in ['train', 'val', 'test']:
            trial_idx_split[split] = trial_idx[np.asarray(f[ split + '_idx'])]
        all_indices[file] =  trial_idx_split
        
np.save('trial_idx_' + mouse + '.npy', all_indices)


_United IDs are same across all files_

In [ ]:
filename = "./" + mouse +'-preproc1.h5'
f = h5py.File(filename, "r")
vox_mask = np.logical_and(np.asarray(f['neurons']['layer'])==b'L2/3', np.asarray(f['neurons']['area'])==b'V1')
np.save('unit_ids_' + mouse + '.npy', np.asarray(f['neurons']['unit_ids'])[vox_mask])

In [ ]:
np.asarray(f['neurons']['unit_ids'])[vox_mask].shape

# Data extraction

In [3]:
mouse = '17358-5-3'

In [4]:
filename = "./" + mouse + "-preproc2.h5"
f = h5py.File(filename, "r")

In [5]:
vox_mask = np.logical_and(np.asarray(f['neurons']['layer'])==b'L2/3', np.asarray(f['neurons']['area'])==b'V1')
vox_mask.sum()

1740

In [6]:
#list_IDs = []
stims = []
resps = []
behav = []
eye = []
for file in [mouse +'-preproc1.h5', mouse +'-preproc2.h5', mouse +'-preproc3.h5']:
    filename = "./" + file 
    with h5py.File(filename, "r") as f:
        train_idx = np.asarray(f['train_idx'])
        train_keys = np.asarray(list(f['inputs'].keys()))[train_idx]
        
        for key in train_keys:
            #print(key)
            img = np.asarray(f['inputs'][key])
            resp = np.asarray(f['responses'][key])[:,vox_mask]
            behav.append(np.asarray(f['behavior'][key]))
            eye.append(np.asarray(f['eye_position'][key]))
            stims.append(img)
            resps.append(resp)
    

# Extract validation data

In [11]:
stims_val = []
resps_val = []
behav_val = []
eye_val = []
for file in [mouse + '-preproc1.h5', mouse + '-preproc2.h5', mouse + '-preproc3.h5']:
    filename = "./" + file 
    with h5py.File(filename, "r") as f:
        val_idx = np.asarray(f['val_idx'])
        val_keys = np.asarray(list(f['inputs'].keys()))[val_idx]
        
        for key in val_keys:
            #print(key)
            img = np.asarray(f['inputs'][key])
            resp = np.asarray(f['responses'][key])[:,vox_mask]
            behav_val.append(np.asarray(f['behavior'][key]))
            eye_val.append(np.asarray(f['eye_position'][key]))   
            stims_val.append(img)
            resps_val.append(resp)

In [12]:
len(behav), behav[0].shape, resp[0].shape

(1101, (450, 3), (4692,))

In [13]:
import pickle
with open('./' + mouse + '.pkl', 'wb') as output:
    pickle.dump({'stims_train': stims, 'responses_train': resps, 'behav_train': behav, 'eye_train': eye,  'stims_val': stims_val, 'responses_val':resps_val, 'behav_val': behav_val, 'eye_val': eye_val}, output)

# Test data

In [14]:
stims_test = []
resps_test = []
behav_test = []
eye_test = []

for file in [mouse + '-preproc1.h5', mouse +  '-preproc2.h5', mouse + '-preproc3.h5']:
    filename = "./" + file 
    with h5py.File(filename, "r") as f:
        test_idx = np.asarray(f['test_idx'])
        test_keys = np.asarray(list(f['inputs'].keys()))[test_idx]
        
        for key in test_keys:
            #print(key)
            img = np.asarray(f['inputs'][key])
            resp = np.asarray(f['responses'][key])#[:,vox_mask]
            behav_test.append(np.asarray(f['behavior'][key]))
            eye_test.append(np.asarray(f['eye_position'][key]))   
            
            stims_test.append(img)
            resps_test.append(resp)

In [15]:
with open('./' + mouse + '_test.pkl', 'wb') as output:
    pickle.dump({'stims_test': stims_test, 'responses_test': resps_test, 'behav_test': behav_test, 'eye_test': eye_test}, output)

In [17]:
mean_resps = np.mean([np.mean(resps[i],0) for i in range(len(resps))],0)
np.save('./' + mouse + '_mean.npy', mean_resps)

In [19]:
eye = np.asarray(eye)
mean_eye = np.mean([np.mean(eye[i],0) for i in range(len(eye))],0)
np.save('./' + mouse + '_mean_eye.npy', mean_eye)

/nfs01/shared_software/anaconda3/envs/meenakshi3/lib/python3.6/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
